# AMPIX data analysis

This script loads data obtained using the AMPIX script. The data should be recorded using the h5AppendRecorder so that each timescan is appended as an entry in a master.h5 file.

The time stamps are in UTC and needs to be converted to human readable time and corrected for time zone. This notebook shows how the time stamps are processed. It further saves the time stamps in a `.txt` file and saved in `/data/visitors/danmax/PROPOSAL/VISIT/process/timestamps/masterfolder/SAMPLE/`

The notebook creates an overview plot and a widget to cycle through individual patterns. The third cell allows for saving of individual patterns as xy-files, but this should not be done on the MaxIV systems, as the thousands of files will cause massive slow down. 

The path to the master.h5 file is required. The timestamp can be written using a date format or in relative time in seconds from the start of the experiment. date_format should be True for a date format or False for a relative time format.

In [31]:
%matplotlib widget
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import glob
from datetime import datetime
import pytz
import re
from ipywidgets import interact
from IPython.display import display


#Input:
fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/masterfolder/SAMPLE/master.h5'
date_format = True

#-----------------------------------------------------------------------------------------------------------------------
fname_file = h5py.File(fname, 'r')

timezone = pytz.timezone('Europe/Stockholm') #The timezone is needed to generate the proper time stamp

azintname = fname.replace('raw','process/azint').split('master')[0]
azintnames = glob.glob(azintname+'*.h5')
azintnames = sorted(azintnames)

I_data = []
stamplist = []
for name in azintnames:
    try:    
        I_file = h5py.File(name, 'r')
        I_data.append(np.mean(I_file['I'], axis=0))
        tth = I_file['2th']
        
        time_name = name.split('_pilatus')[0]        
        entry_number = name.split('_pilatus')[0][-4:]
        timeStampUnix = fname_file['/entry'+entry_number+'/measurement/pcap_trigts'][0]
        offsetTZ = timezone.localize(datetime.utcfromtimestamp(timeStampUnix)).utcoffset()
        stamp = datetime.utcfromtimestamp(timeStampUnix)+offsetTZ
        stamplist.append(stamp)
        
       
        
    except:
        I_data.append(np.zeros_like(tth))
        print(name) #This should print the name of files that have errors.


path, fileName = os.path.split(fname)
out_path = path.replace('raw', 'process/timestamps')
if not os.path.exists(out_path):
    print(f'Creating folder: {out_path}')
    os.makedirs(out_path)         
out = os.path.join(out_path,fileName.split('.')[0]+'_timestamps.txt')

with open(out, "w+") as f:  
    for time in stamplist:
        if date_format:
            f.write(str(time)+'\n')
        else:
            t0 = stamplist[0]
            dt = time-t0
            f.write(str(dt.seconds)+'\n')
              
tth = np.array(tth)
I_data = np.array(I_data)

plt.figure()
plt.title('PC86')
plt.imshow(I_data, extent=[tth[0], tth[-1], 0, I_data.shape[0]], interpolation = 'Nearest', aspect = 'auto', norm=LogNorm(), origin = 'lower')
plt.xlabel('2th')
plt.ylabel('Scan number')

/data/visitors/danmax/20211101/2022042908/process/azint/Run2/S2_1/scan-0162_pilatus_integrated.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Scan number')

In [2]:
#Widget
plt.figure()
def plotting(frame):
    plt.clf()
    plt.plot(tth, I_data[frame, :])
    #plt.xlim([3, 6.5]) #Specify a 2theta range.
    #plt.ylim([0, 375]) #Specify an intensity range.
interact(plotting, frame=(0, len(I_data)-1, 1)) #The numbers in the frame bracket are start frame, end frame and spacing, respectively. If the spacing number is 1, all frames will be shown: if it
#is 20, every 20 frames will be shown.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=173, description='frame', max=346), Output()), _dom_classes=('widget-int…

<function __main__.plotting(frame)>

In [23]:
#Save xy-data. Please be aware that a large number of files can be generated. Only save to your personal computer! Don't save it in the DanMAX folders! This only serves as inspiration 
#for how to save the files on your own system.

integrated_folder = fname.replace('raw','process/patterns').split('master')[0]
if not os.path.exists(integrated_folder):
    print(f'Creating folder: {integrated_folder}')
    os.makedirs(integrated_folder) 

for scan in range(I_data.shape[0]):
    #integrated_name = azint_fname.split('.')[0] + '_scan_{}'.format(no)+'.xy'

    integrated_name = integrated_folder + 'scan_{}'.format(scan)+'.xy'
    np.savetxt(integrated_name, np.column_stack((tth, I_data[scan, :])))

Creating folder: /data/visitors/danmax/20211101/2022042908/process/patterns/Run2/S2_1/
